In [1]:
!pip install simpletransformers

     |████████████████████████████████| 249 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 23.1 MB/s eta 0:00:01
     |████████████████████████████████| 5.9 MB 51.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 67.8 MB/s eta 0:00:01
     |████████████████████████████████| 9.1 MB 72.8 MB/s eta 0:00:01
     |████████████████████████████████| 365 kB 71.5 MB/s eta 0:00:01
  Using cached pathtools-0.1.2-py3-none-any.whl
     |████████████████████████████████| 157 kB 65.7 MB/s eta 0:00:01
  Using cached promise-2.3-py3-none-any.whl
     |████████████████████████████████| 281 kB 120.1 MB/s eta 0:00:01
  Using cached GitPython-3.1.27-py3-none-any.whl (181 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached gitdb-4.0.9-py3-none-any.whl (63 kB)
  Using cached smmap-5.0.0-py3-none-any.whl (24 kB)
     |████████████████████████████████| 140 kB 56.1 MB/s eta 0:00:01
  Using cached xxhash-3.0.0-cp39-cp39-manylinux_2_17_x86_64

  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
  Using cached semver-2.13.0-py2.py3-none-any.whl (12 kB)
  Using cached altair-4.2.0-py3-none-any.whl (812 kB)
     |████████████████████████████████| 235 kB 30.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 69.4 MB/s eta 0:00:01
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
     |████████████████████████████████| 4.7 MB 115.2 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 15.2 MB/s  eta 0:00:01
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
     |████████████████████████████████| 55 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 51 kB 7.2 MB/s  eta 0:00:01
  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
     |████████████████████████████████| 336 kB 21.2 MB/s eta 0:00:01
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached protobuf-3.19.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB

In [2]:
import pandas as pd
import math
import numpy as np
from seqeval.metrics import f1_score

from seqeval.metrics import classification_report, accuracy_score, f1_score
from tqdm import tqdm,trange

In [3]:
!pwd

/home/admin-user/swardi/droner


In [4]:
dataset_train_path = 'droner_train_2.csv'
dataset_test_path = 'droner_test_2.csv'
dataset_train = pd.read_csv(dataset_train_path, encoding= 'unicode_escape')
dataset_test = pd.read_csv(dataset_test_path, encoding= 'unicode_escape')
dataset_train

,Unnamed: 0,drone_model,dataset,controller,sentence_id,words,labels
0,0,DJI_Inspire_1,df010,mobile_iOS,1,Taking,B-ACTION
1,1,DJI_Inspire_1,df010,mobile_iOS,1,Off,I-ACTION
2,2,DJI_Inspire_1,df010,mobile_iOS,1,.,O
3,3,DJI_Inspire_1,df010,mobile_iOS,2,Taking,B-ACTION
4,4,DJI_Inspire_1,df010,mobile_iOS,2,Off,I-ACTION
...,...,...,...,...,...,...,...
8645,8645,DJI_Mavic_Pro,df021,mobile_iOS_backup,1395,be,O
8646,8646,DJI_Mavic_Pro,df021,mobile_iOS_backup,1395,disabled,O
8647,8647,DJI_Mavic_Pro,df021,mobile_iOS_backup,1395,in,O
8648,8648,DJI_Mavic_Pro,df021,mobile_iOS_backup,1395,landing,O


In [5]:
dataset_train = dataset_train.iloc[:, [4,5,6]]
dataset_train

,sentence_id,words,labels
0,1,Taking,B-ACTION
1,1,Off,I-ACTION
2,1,.,O
3,2,Taking,B-ACTION
4,2,Off,I-ACTION
...,...,...,...
8645,1395,be,O
8646,1395,disabled,O
8647,1395,in,O
8648,1395,landing,O


In [6]:
dataset_test = dataset_test.iloc[:, [4,5,6]]

In [7]:
dataset_train["labels"].value_counts()

O              4590
I-PARAMETER     964
B-PARAMETER     884
B-ISSUE         681
B-COMPONENT     314
I-FUNCTION      271
B-FUNCTION      217
I-ISSUE         212
B-ACTION        183
I-COMPONENT     144
I-ACTION         98
B-STATE          48
I-STATE          44
Name: labels, dtype: int64

In [8]:
dataset_test["labels"].value_counts()

O              2503
I-PARAMETER     167
I-ISSUE         143
B-PARAMETER     139
B-COMPONENT     132
B-ISSUE         101
B-ACTION        100
I-FUNCTION       86
B-FUNCTION       77
B-STATE          71
I-STATE          48
I-ACTION         25
I-COMPONENT      15
Name: labels, dtype: int64

In [9]:
label = dataset_train["labels"].unique().tolist()
label

['B-ACTION',
 'I-ACTION',
 'O',
 'B-PARAMETER',
 'I-PARAMETER',
 'B-STATE',
 'I-STATE',
 'B-COMPONENT',
 'B-ISSUE',
 'I-ISSUE',
 'B-FUNCTION',
 'I-FUNCTION',
 'I-COMPONENT']

In [10]:
from simpletransformers.ner import NERModel, NERArgs

args = NERArgs()
args.num_train_epochs = 4
args.learning_rate = 0.0001 # 0.0001  5e-5, 3e-5, 2e-5
args.overwrite_output_dir = True
args.train_batch_size = 16
args.eval_batch_size = 16

In [11]:
model = NERModel("bert", "bert-base-cased", labels=label, args =args, use_cuda=True)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [13]:
model.train_model(dataset_train, eval_data = dataset_test, acc=accuracy_score)

  0%|          | 0/3 [00:00<?, ?it/s]

/home/admin-user/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/86 [00:00<?, ?it/s]

/home/admin-user/miniconda3/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 4:   0%|          | 0/86 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/86 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/86 [00:00<?, ?it/s]

(344, 0.12641687854869985)

In [14]:
result, model_outputs, preds_list = model.eval_model(dataset_test)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/26 [00:00<?, ?it/s]

In [15]:
result

{'eval_loss': 0.25158813345590786,
 'precision': 0.8252730109204368,
 'recall': 0.808868501529052,
 'f1_score': 0.8169884169884171}

In [16]:
# Groupby and collect columns
dataset_test_group = dataset_test.groupby(['sentence_id'], as_index=False)['words', 'labels'].agg(lambda x: list(x))
y_test = dataset_test_group['labels']
y_test

/tmp/ipykernel_26923/1180599873.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dataset_test_group = dataset_test.groupby(['sentence_id'], as_index=False)['words', 'labels'].agg(lambda x: list(x))


0              [B-PARAMETER, I-PARAMETER, O, O, B-STATE]
1      [O, B-PARAMETER, I-PARAMETER, O, B-STATE, O, O...
2              [B-PARAMETER, I-PARAMETER, O, O, B-STATE]
3      [O, B-PARAMETER, I-PARAMETER, O, B-STATE, O, O...
4                       [B-PARAMETER, I-PARAMETER, O, O]
                             ...                        
402           [B-FUNCTION, I-FUNCTION, O, O, O, O, O, O]
403    [O, O, B-COMPONENT, O, O, O, O, B-ACTION, O, O...
404    [B-PARAMETER, I-PARAMETER, O, O, B-PARAMETER, ...
405    [O, O, O, O, O, O, B-COMPONENT, O, O, O, O, O,...
406    [B-COMPONENT, O, O, B-PARAMETER, I-PARAMETER, ...
Name: labels, Length: 407, dtype: object

In [17]:
y_pred_list = []
y_test_list = []
for row in range(0, len(preds_list)):
  y_pred_list = np.concatenate((y_pred_list, preds_list[row]), axis=0)
  y_test_list = np.concatenate((y_test_list, y_test[row]), axis=0)
y_pred_list

array(['B-PARAMETER', 'I-PARAMETER', 'O', ..., 'O', 'O', 'O'],
      dtype='<U32')

In [18]:
data_prediction = pd.DataFrame({'actual_class': y_test_list, 'predicted_class': y_pred_list})
data_prediction

,actual_class,predicted_class
0,B-PARAMETER,B-PARAMETER
1,I-PARAMETER,I-PARAMETER
2,O,O
3,O,O
4,B-STATE,B-PARAMETER
...,...,...
3602,O,O
3603,O,O
3604,O,O
3605,O,O


In [19]:
confusion_matrix = pd.crosstab(data_prediction['predicted_class'], data_prediction['actual_class'])
confusion_matrix

actual_class,B-ACTION,B-COMPONENT,B-FUNCTION,B-ISSUE,B-PARAMETER,B-STATE,I-ACTION,I-COMPONENT,I-FUNCTION,I-ISSUE,I-PARAMETER,I-STATE,O
predicted_class,,,,,,,,,,,,,
B-ACTION,51,0,3,0,0,1,0,0,0,0,0,0,6
B-COMPONENT,0,111,0,2,0,0,0,0,0,0,0,0,1
B-FUNCTION,0,0,69,0,0,0,0,0,0,0,0,0,2
B-ISSUE,20,0,3,99,0,0,0,0,0,1,0,0,0
B-PARAMETER,0,0,0,0,138,40,0,0,0,0,0,0,14
B-STATE,0,0,0,0,0,30,0,0,0,0,0,0,0
I-ACTION,0,0,0,0,0,0,25,0,0,0,0,0,0
I-COMPONENT,0,0,0,0,0,0,0,15,3,0,0,0,0
I-FUNCTION,1,0,2,0,0,0,0,0,74,0,0,0,0


In [20]:
y_test = data_prediction['actual_class'].to_list()
y_pred = data_prediction['predicted_class'].to_list()

In [21]:
from sklearn.metrics import classification_report
pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, labels=['O', 'B-ACTION', 'I-ACTION', 'B-COMPONENT', 'I-COMPONENT', 'B-FUNCTION', 'I-FUNCTION', 'B-ISSUE', 'I-ISSUE', 'B-STATE', 'I-STATE', 'B-PARAMETER', 'I-PARAMETER'])).T

,precision,recall,f1-score,support
O,0.977111,0.989213,0.983125,2503.000000
B-ACTION,0.836066,0.510000,0.633540,100.000000
I-ACTION,1.000000,1.000000,1.000000,25.000000
B-COMPONENT,0.973684,0.840909,0.902439,132.000000
I-COMPONENT,0.833333,1.000000,0.909091,15.000000
B-FUNCTION,0.971831,0.896104,0.932432,77.000000
I-FUNCTION,0.961039,0.860465,0.907975,86.000000
B-ISSUE,0.804878,0.980198,0.883929,101.000000
I-ISSUE,0.979310,0.993007,0.986111,143.000000
B-STATE,1.000000,0.422535,0.594059,71.000000


In [22]:
from seqeval.metrics import classification_report
pd.DataFrame(classification_report([y_test], [y_pred], output_dict=True)).T

,precision,recall,f1-score,support
ACTION,0.784615,0.510000,0.618182,100.0
COMPONENT,0.948718,0.840909,0.891566,132.0
FUNCTION,0.959459,0.922078,0.940397,77.0
ISSUE,0.784000,0.970297,0.867257,101.0
PARAMETER,0.730594,0.993789,0.842105,161.0
STATE,0.926829,0.457831,0.612903,83.0
micro avg,0.825273,0.808869,0.816988,654.0
macro avg,0.855703,0.782484,0.795402,654.0
weighted avg,0.842977,0.808869,0.804218,654.0


In [23]:
prediction, model_output = model.predict(["Aircraft is returning to the Home Point Minimum RTH Altitude is 30m You can reset the RTH Altitude in Remote Controller Settings after cancelling RTH"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
prediction

[[{'Aircraft': 'B-COMPONENT'},
  {'is': 'O'},
  {'returning': 'O'},
  {'to': 'O'},
  {'the': 'O'},
  {'Home': 'B-PARAMETER'},
  {'Point': 'I-PARAMETER'},
  {'Minimum': 'B-PARAMETER'},
  {'RTH': 'I-PARAMETER'},
  {'Altitude': 'I-PARAMETER'},
  {'is': 'O'},
  {'30m': 'O'},
  {'You': 'O'},
  {'can': 'O'},
  {'reset': 'O'},
  {'the': 'O'},
  {'RTH': 'B-PARAMETER'},
  {'Altitude': 'I-PARAMETER'},
  {'in': 'O'},
  {'Remote': 'B-FUNCTION'},
  {'Controller': 'I-FUNCTION'},
  {'Settings': 'I-FUNCTION'},
  {'after': 'O'},
  {'cancelling': 'O'},
  {'RTH': 'B-ACTION'}]]

In [25]:
prediction2, model_output2 = model.predict(["Weak signal Avoid blocking the antennas and keep the antennas parallel to and facing toward the aircraft during flight"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
prediction2

[[{'Weak': 'B-ISSUE'},
  {'signal': 'I-ISSUE'},
  {'Avoid': 'O'},
  {'blocking': 'O'},
  {'the': 'O'},
  {'antennas': 'B-COMPONENT'},
  {'and': 'O'},
  {'keep': 'O'},
  {'the': 'O'},
  {'antennas': 'B-COMPONENT'},
  {'parallel': 'O'},
  {'to': 'O'},
  {'and': 'O'},
  {'facing': 'O'},
  {'toward': 'O'},
  {'the': 'O'},
  {'aircraft': 'B-COMPONENT'},
  {'during': 'O'},
  {'flight': 'O'}]]

In [27]:
model.push_to_hub("droner-cosine")

AttributeError: 'NERModel' object has no attribute 'push_to_hub'